In [1]:
import numpy as np
import itertools
import os
import sys
sys.path.append(os.path.expanduser('~/smodels'))
from smodels.tools.xsecComputer import XSecComputer, LO
from smodels.tools.physicsUnits import TeV

In [2]:
mHSCP = np.linspace(25.,3000.,100)

In [3]:
# columns = ('msl3','Wsl3')
columns = ['msl3']
outputFile = 'THSCPM1b_pts_validation.txt'
header = '%19s'*len(columns) %tuple(columns)
header = header[3:]
np.savetxt(outputFile,mHSCP,header=header,fmt = ['     %1.7e']*len(columns))

In [4]:
template = 'THSCPM1b_template.slha'
with open(template,'r') as f:
    lines = f.readlines()
iw = [i for i,l in enumerate(lines) if 'DECAY 1000015 1.000000e-30 # Wsl3' in l][0]
im = [i for i,l in enumerate(lines) if '1000015 1.00000000E+02 # Msl3' in l][0]


widths = np.logspace(-25,-15,30)
computer = XSecComputer( LO, 1000, pythiaVersion=8, maycompile=False)
for mass in mHSCP:
    slhaFile = './slha/THSCPM1b_%1.0f.slha' %(mass)
    lines[im] = '  1000015 %1.8E # Msl3 \n' %mass
    with open(slhaFile,'w') as f:
        for l in lines: f.write(l)
            
    xsecs = computer.compute( 8*TeV, slhaFile, unlink=True,
                                    loFromSlha=False, pythiacard= './pythia8_stau.cfg')
    add = computer.addXSecToFile(xsecs,slhaFile)

    for width in widths:
        with open(slhaFile,'r') as ff:
            newLines = ff.readlines()
        
        newFile = './slha/THSCPM1b_%1.0f_%1.2e.slha' %(mass,width)
        newLines[iw] = 'DECAY 1000015 %1.6e # Wsl3 \n' %width
        with open(newFile,'w') as f:
            for l in newLines: f.write(l)
    os.remove(slhaFile)

In [6]:
print(widths.tolist())

[1e-25, 2.21221629107045e-25, 4.893900918477499e-25, 1.0826367338740562e-24, 2.3950266199874907e-24, 5.2983169062837234e-24, 1.1721022975334842e-23, 2.5929437974046775e-23, 5.736152510448705e-23, 1.2689610031679285e-22, 2.8072162039411697e-22, 6.210169418915604e-22, 1.3738237958832609e-21, 3.0391953823131947e-21, 6.723357536499335e-21, 1.487352107293512e-20, 3.290344562312671e-20, 7.278953843983161e-20, 1.6102620275609425e-19, 3.5622478902624516e-19, 7.880462815669937e-19, 1.7433288221999944e-18, 3.856620421163457e-18, 8.531678524172779e-18, 1.887391822135092e-17, 4.175318936560392e-17, 9.236708571873846e-17, 2.043359717856948e-16, 4.52035365636024e-16, 1e-15]
